In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


import pandas as pd
import numpy as np
import cv2
from skimage import exposure
from scipy.ndimage import gaussian_filter
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import LearningRateScheduler
import math
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import pandas as pd
import cv2
from skimage import exposure
from scipy.ndimage import gaussian_filter
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [2]:
# Load the dataset
data = pd.read_csv("test_split_image_key.csv")

# Split the dataset into training and testing sets
train_data = data[data["split_type"] == "training"]
test_data = data[data["split_type"] == "testing"]

In [3]:
retinopathy_grade_dtype = data["Retinopathy grade"].dtype
print("Retinopathy Grade Data Type:", retinopathy_grade_dtype)

Retinopathy Grade Data Type: int64


In [4]:
# Function to load images as they are, without preprocessing
def load_image(image_path):
    # Load and resize the image to 224 x 224
    image = cv2.imread(image_path)
    resized_image = cv2.resize(image, (224, 224))
    return resized_image


# Load training and testing images into numpy arrays
X_train = [
    load_image("a_training/" + img_name + ".jpg")
    for img_name in train_data["Image name"]
]
X_test = [
    load_image("b_testing/" + img_name + ".jpg") for img_name in test_data["Image name"]
]

X_train = np.array(X_train)
X_test = np.array(X_test)

# Extract labels directly without one-hot encoding
y_train = train_data["Retinopathy grade"].values
y_test = test_data["Retinopathy grade"].values

> Initialize base model with pre-trained weights 

In [5]:
base_model = tf.keras.applications.EfficientNetB0(
    include_top=False,
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_tensor=None,
    input_shape=(224, 224, 3),
    pooling="avg",
)

> freeze the base model

In [6]:
base_model.trainable = False

> Create a new model on top with dropout

In [7]:
num_classes = 5
epochs = 50
# Define the inputs
inputs = tf.keras.Input(shape=(224, 224, 3))

# Ensure the base_model is running in inference mode.
x = base_model(inputs, training=False)

x = layers.Dropout(0.5)(x)  # Dropout layer with 50% dropout rate
# Adding FC (Fully Connected) layers
x = layers.Dense(1000)(x)
x = layers.Dropout(0.5)(x)  # Another Dropout layer with 50% dropout rate
# Adding FC (Fully Connected) layers
x = layers.Dense(500)(x)

# Adding a final layer with SoftMax activation for classification
outputs = layers.Dense(num_classes, activation="softmax")(x)

# Creating the model
model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.summary(show_trainable=True)

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┓
┃ Layer (type)                ┃ Output Shape          ┃    Param # ┃ Trai… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━┩
│ input_layer_1 (InputLayer)  │ (None, 224, 224, 3)   │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ efficientnetb0 (Functional) │ (None, 1280)          │  4,049,571 │   N   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dropout (Dropout)           │ (None, 1280)          │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense (Dense)               │ (None, 1000)          │  1,281,000 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dropout_1 (Dropout)         │ (None, 1000)          │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense_1 (Dense)             │ (None, 500)           │    500,500 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense_2 (Dense)             │ (None, 5)             │      2,505 │   Y   │
└─────────────────────────────┴───────────────────────┴────────────┴───────┘

 Total params: 5,833,576 (22.25 MB)

 Trainable params: 1,784,005 (6.81 MB)

 Non-trainable params: 4,049,571 (15.45 MB)

> Train the model on new data

In [8]:
from tensorflow.keras.optimizers import SGD

# base learning rate
base_learning_rate = 1e-4
# maximum learning rate
max_learning_rate = 1e-2

# Create an instance of SGD optimizer with initial learning rate
optimizer = SGD(learning_rate=base_learning_rate, momentum=0.9)

# create class weight
classes = np.unique(y_train)
class_weights = compute_class_weight("balanced", classes=classes, y=y_train)
class_weight_dict = dict(zip(classes, class_weights))


# create triangular schedule
def triangular_schedule(epoch):
    """Triangular learning rate scheduler."""
    cycle_length = 10  # Define the length of a cycle
    cycle = math.floor(1 + epoch / (2 * cycle_length))
    x = abs(epoch / cycle_length - 2 * cycle + 1)
    lr = base_learning_rate + (max_learning_rate - base_learning_rate) * max(0, (1 - x))
    return lr


# When fitting the model, include the learning rate scheduler callback
lr_scheduler = LearningRateScheduler(triangular_schedule)

In [9]:
model.compile(
    optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

print("Fitting the top layer of the model")
model.fit(
    X_train,
    y_train,
    epochs=epochs,
    batch_size=32,
    class_weight=class_weight_dict,
    callbacks=[lr_scheduler],
)

Fitting the top layer of the model
Epoch 1/50


13/13 ━━━━━━━━━━━━━━━━━━━━ 9s 410ms/step - accuracy: 0.2043 - loss: 2.0630 - learning_rate: 1.0000e-04
Epoch 2/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 403ms/step - accuracy: 0.2267 - loss: 1.9407 - learning_rate: 0.0011
Epoch 3/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 397ms/step - accuracy: 0.3613 - loss: 1.5451 - learning_rate: 0.0021
Epoch 4/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 378ms/step - accuracy: 0.3850 - loss: 1.5097 - learning_rate: 0.0031
Epoch 5/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 409ms/step - accuracy: 0.3406 - loss: 1.5060 - learning_rate: 0.0041
Epoch 6/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 386ms/step - accuracy: 0.3355 - loss: 1.5134 - learning_rate: 0.0050
Epoch 7/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 406ms/step - accuracy: 0.4255 - loss: 1.2878 - learning_rate: 0.0060
Epoch 8/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 405ms/step - accuracy: 0.4322 - loss: 1.2335 - learning_rate: 0.0070
Epoch 9/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 402ms/step - accuracy: 0.4374 - loss: 1.5818 - learning_rate: 0.0080
Epoch 10/50
13/13

> Fine tuning: unfreeze all or part of the base model and retrain the whole model end-to-end with a very low learning rate.

In [10]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Unfreeze the base model
base_model.trainable = True
model.summary(show_trainable=True)

# Re-instantiate the optimizer
optimizer = SGD(learning_rate=base_learning_rate, momentum=0.9)

model.compile(
    optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

print("Fitting the end-to-end model")
# Train end-to-end. Stop before overfit
model.fit(
    X_train,
    y_train,
    epochs=epochs,
    batch_size=32,
    class_weight=class_weight_dict,
    callbacks=[
        lr_scheduler,
    ],
)

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┓
┃ Layer (type)                ┃ Output Shape          ┃    Param # ┃ Trai… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━┩
│ input_layer_1 (InputLayer)  │ (None, 224, 224, 3)   │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ efficientnetb0 (Functional) │ (None, 1280)          │  4,049,571 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dropout (Dropout)           │ (None, 1280)          │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense (Dense)               │ (None, 1000)          │  1,281,000 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dropout_1 (Dropout)         │ (None, 1000)          │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense_1 (Dense)             │ (None, 500)           │    500,500 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense_2 (Dense)             │ (None, 5)             │      2,505 │   Y   │
└─────────────────────────────┴───────────────────────┴────────────┴───────┘

 Total params: 7,617,583 (29.06 MB)

 Trainable params: 5,791,553 (22.09 MB)

 Non-trainable params: 42,023 (164.16 KB)

 Optimizer params: 1,784,007 (6.81 MB)

Fitting the end-to-end model
Epoch 1/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 52s 3s/step - accuracy: 0.2855 - loss: 1.6575 - learning_rate: 1.0000e-04
Epoch 2/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 42s 3s/step - accuracy: 0.2993 - loss: 1.5589 - learning_rate: 0.0011
Epoch 3/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 54s 4s/step - accuracy: 0.4041 - loss: 1.2903 - learning_rate: 0.0021
Epoch 4/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 46s 4s/step - accuracy: 0.4362 - loss: 1.0729 - learning_rate: 0.0031
Epoch 5/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 44s 3s/step - accuracy: 0.5126 - loss: 0.9374 - learning_rate: 0.0041
Epoch 6/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 44s 3s/step - accuracy: 0.6447 - loss: 0.7271 - learning_rate: 0.0050
Epoch 7/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 45s 3s/step - accuracy: 0.6550 - loss: 0.6703 - learning_rate: 0.0060
Epoch 8/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 44s 3s/step - accuracy: 0.7967 - loss: 0.4998 - learning_rate: 0.0070
Epoch 9/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 46s 4s/step - accuracy: 0.7761 - loss: 0.4945 - learning_rate: 0.

> Evaluate on test data

In [11]:
print("Test dataset evaluation")
model.evaluate(X_test, y_test)

Test dataset evaluation
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 411ms/step - accuracy: 0.5608 - loss: 2.8218


[3.0512337684631348, 0.5242718458175659]